# Horizontal

In [11]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def extract_articles_from_url2(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)

        # Obtener todos los artículos dentro de las etiquetas <article>
        articles = driver.find_elements(By.CSS_SELECTOR, ".row article")

        data = []
        for article in articles:
            # Imagen
            try:
                img = article.find_element(By.CSS_SELECTOR, "a img")
                image_url = img.get_attribute("src")
            except:
                image_url = None
                
            # Título y enlace
            try:
                title = article.find_element(By.CSS_SELECTOR, ".tarjeta__titulo a").text.strip()
                link = article.find_element(By.CSS_SELECTOR, ".tarjeta__titulo a").get_attribute("href")
            except:
                continue  # Si no hay título o enlace, saltamos este artículo

            # Autor
            try:
                author = article.find_element(By.CSS_SELECTOR, ".tarjeta__autor a").text.strip()
            except:
                author = "No especificado"
            
            # Fecha
            try:
                date = article.find_element(By.CSS_SELECTOR, ".tarjeta-fecha__texto").text.strip()
            except:
                date = "No especificada"
            
            # Contenido (extracto del texto)
            try:
                content = article.find_element(By.CSS_SELECTOR, ".tarjeta__texto").text.strip()
            except:
                content = None

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Autor': author,
                'Fecha Publicación': date,
                'Categorías': 'Estudios',  # Campo fijo basado en el ejemplo
                'URL Imagen': image_url,
                'Contenido': content,
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

def extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los artículos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana). Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)  # Inicializar el WebDriver solo una vez
    current_page = start_page
    all_articles_df = pd.DataFrame()  # Inicializamos un DataFrame vacío

    while current_page <= max_pages:
        # Construir la URL de la página actual
        url = f"{base_url}{current_page}/"  # Aquí el formato de la URL, el número de página se añade al final
        print(f"Extrayendo artículos de: {url}")

        # Llamar a la función extract_articles_from_url2 para obtener artículos de la página
        all_articles_df = extract_articles_from_url2(driver, url, all_articles_df)

        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_articles_df is None or len(all_articles_df) == 0:
            print(f"No se encontraron artículos en la página {current_page}, o la página no existe.")
            break

        current_page += 1  # Pasar a la siguiente página

    driver.quit()  # Cerrar el WebDriver después de todas las iteraciones
    return all_articles_df

In [14]:
base_url = "https://horizontalchile.cl/publicaciones/page/"
all_articles_df = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=32, headless=True)

Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/1/
Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/2/
Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/3/
Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/4/
Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/5/
Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/6/
Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/7/
Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/8/
Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/9/
Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/10/
Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/11/
Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/12/
Extrayendo artículos de: https://horizontalchile.cl/publicaciones/page/13/
Extrayendo artículos de: https://h

In [15]:
all_articles_df

,Título,Enlace,Autor,Fecha Publicación,Categorías,URL Imagen,Contenido,Fecha Consulta
0,El estallido social chileno visto como un «eve...,https://horizontalchile.cl/publicacion/el-esta...,Juan Luis Ossa,18 Oct 2024,Estudios,https://horizontalchile.cl/assets/uploads/2024...,"A cinco años del estallido social, el document...",2024-11-25 01:20:46
1,Análisis del presupuesto en seguridad 2024-2025,https://horizontalchile.cl/publicacion/analisi...,Alfonso España,17 Oct 2024,Estudios,https://horizontalchile.cl/assets/uploads/2024...,El siguiente documento hace una revisión del n...,2024-11-25 01:20:46
2,Financiamiento Público de la Educación Superio...,https://horizontalchile.cl/publicacion/financi...,Santiago Montiel,15 Oct 2024,Estudios,https://horizontalchile.cl/assets/uploads/2024...,Según estimaciones del centro de estudios Hori...,2024-11-25 01:20:46
3,Aprender por Chile,https://horizontalchile.cl/publicacion/aprende...,No especificado,14 Oct 2024,Estudios,https://horizontalchile.cl/assets/uploads/2024...,Los principales aprendizajes que nos dejan est...,2024-11-25 01:20:46
4,Análisis crítico de la ley de estabilización d...,https://horizontalchile.cl/publicacion/analisi...,Soledad Hormazábal,23 Sep 2024,Estudios,https://horizontalchile.cl/assets/uploads/2024...,Resulta muy preocupante que se realice polític...,2024-11-25 01:20:46
...,...,...,...,...,...,...,...,...
178,Comparación Internacional del Índice de Opor...,https://horizontalchile.cl/publicacion/compara...,No especificado,04 Mar 2013,Estudios,https://horizontalchile.cl/assets/uploads/2013...,"A continuación, presentamos la primera aplicac...",2024-11-25 01:23:18
179,Índice de oportunidades para la movilidad social,https://horizontalchile.cl/publicacion/indice-...,No especificado,03 Mar 2013,Estudios,https://horizontalchile.cl/assets/uploads/2013...,Si los talentos innatos de un niño son indepen...,2024-11-25 01:23:18
180,Desigualdades laborales que impactan a futuro,https://horizontalchile.cl/publicacion/desigua...,No especificado,02 Mar 2013,Estudios,https://horizontalchile.cl/assets/uploads/2013...,En este documento presentamos un análisis de l...,2024-11-25 01:23:22
181,Radiografía al Mercado Laboral femenino en Chile,https://horizontalchile.cl/publicacion/radiogr...,No especificado,05 Ene 2013,Estudios,https://horizontalchile.cl/assets/uploads/2013...,Este documento corresponde a un análisis de la...,2024-11-25 01:23:22


## Iteración por enlaces

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (se puede ajustar el tiempo de espera si es necesario)
    driver.implicitly_wait(3)  # Esperar hasta 3 segundos para que se cargue la página
    
    # Extraer la información de interés:
    
    try:
        title = driver.find_element(By.CSS_SELECTOR, "h1.text-center").text.strip()
    except:
        title = "No disponible"
    
    try:
        author = driver.find_element(By.CSS_SELECTOR, ".noticia-info__autor a").text.strip()
    except:
        author = "No disponible"
    
    try:
        publication_date = driver.find_element(By.CSS_SELECTOR, ".noticia-info__texto").text.strip()
    except:
        publication_date = "No disponible"
    
    try:
        categories = driver.find_element(By.CSS_SELECTOR, ".noticia-info__link").text.strip()
    except:
        categories = "No disponible"
    
    try:
        content = driver.find_element(By.CSS_SELECTOR, ".contenido").text.strip()
    except:
        content = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Autor": author,
        "Fecha de Publicación": publication_date,
        "Categorías": categories,
        "Contenido": content,
        "Enlace": url,
    }
    
    return article_data

def process_all_links(jg_df, headless=True):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.
    
    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.
        headless (bool): Determina si el modo headless debe estar activado o no. Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        for _, row in jg_df.iterrows():
            url = row['Enlace']
            print(f"Extrayendo datos de: {url}")
            
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar


In [20]:
all_articles_df2 = all_articles_df[:3]
horizontal_df = process_all_links(all_articles_df, headless=False)
horizontal_df

Extrayendo datos de: https://horizontalchile.cl/publicacion/el-estallido-social-chileno-visto-como-un-evento-historico-una-mirada-desde-la-centroderecha/
Extrayendo datos de: https://horizontalchile.cl/publicacion/analisis-del-presupuesto-en-seguridad-2024-2025/
Extrayendo datos de: https://horizontalchile.cl/publicacion/financiamiento-publico-de-la-educacion-superior-quien-paga-el-impuesto/
Extrayendo datos de: https://horizontalchile.cl/publicacion/aprender-por-chile/
Extrayendo datos de: https://horizontalchile.cl/publicacion/analisis-critico-de-la-ley-de-estabilizacion-de-tarifas-electricas-y-subsidio-electrico/
Extrayendo datos de: https://horizontalchile.cl/publicacion/two-journeys-twice-the-learning-10-lessons-from-the-chilean-constitutional-experience/
Extrayendo datos de: https://horizontalchile.cl/publicacion/dos-viajes-el-doble-de-aprendizajes-10-lecciones-de-la-experiencia-constitucional-chilena/
Extrayendo datos de: https://horizontalchile.cl/publicacion/perspectiva-horizo

,Título,Autor,Fecha de Publicación,Categorías,Contenido,Enlace
0,"El estallido social chileno visto como un ""eve...",Juan Luis Ossa,18 Oct 2024,Estudios,El 18 de octubre de 2019 fue un evento históri...,https://horizontalchile.cl/publicacion/el-esta...
1,Análisis del presupuesto en seguridad 2024-2025,Alfonso España,17 Oct 2024,Estudios,El siguiente documento hace una revisión del n...,https://horizontalchile.cl/publicacion/analisi...
2,Financiamiento Público de la Educación Superio...,Santiago Montiel,15 Oct 2024,Estudios,La lógica de impuesto que presenta el Financia...,https://horizontalchile.cl/publicacion/financi...
3,Aprender por Chile,No disponible,14 Oct 2024,Estudios,Hace cinco años nuestra democracia estuvo en g...,https://horizontalchile.cl/publicacion/aprende...
4,Análisis crítico de la ley de estabilización d...,Soledad Hormazábal,23 Sep 2024,Estudios,El proyecto en cuestión puede poner en riesgo ...,https://horizontalchile.cl/publicacion/analisi...
...,...,...,...,...,...,...
178,Comparación Internacional del Índice de Opor...,No disponible,04 Mar 2013,Estudios,"A continuación, presentamos la primera aplicac...",https://horizontalchile.cl/publicacion/compara...
179,Índice de oportunidades para la movilidad social,No disponible,03 Mar 2013,Estudios,Si los talentos innatos de un niño son indepen...,https://horizontalchile.cl/publicacion/indice-...
180,Desigualdades laborales que impactan a futuro,No disponible,02 Mar 2013,Estudios,Las desigualdades previsionales entre hombres ...,https://horizontalchile.cl/publicacion/desigua...
181,Radiografía al Mercado Laboral femenino en Chile,No disponible,05 Ene 2013,Estudios,Este documento corresponde a un análisis de la...,https://horizontalchile.cl/publicacion/radiogr...


In [21]:
horizontal_df.to_excel("Horizontal.xlsx", index = False)

# Instituto Libertad

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def extract_articles_from_url2(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)

        # Obtener todos los artículos dentro de las etiquetas <article>
        articles = driver.find_elements(By.CSS_SELECTOR, ".elementor-post")

        data = []
        for article in articles:
            # Imagen
            try:
                img = article.find_element(By.CSS_SELECTOR, ".elementor-post__thumbnail img")
                image_url = img.get_attribute("src")
            except:
                image_url = None
                
            # Título y enlace
            try:
                title = article.find_element(By.CSS_SELECTOR, ".elementor-post__title a").text.strip()
                link = article.find_element(By.CSS_SELECTOR, ".elementor-post__title a").get_attribute("href")
            except:
                continue  # Si no hay título o enlace, saltamos este artículo

            # Autor (esto no parece estar directamente disponible en el HTML proporcionado, se deja como "No especificado")
            author = "No especificado"
            
            # Fecha (el selector de la fecha está en .elementor-post__meta-data .elementor-post-date)
            try:
                date = article.find_element(By.CSS_SELECTOR, ".elementor-post__meta-data .elementor-post-date").text.strip()
            except:
                date = "No especificada"
            
            # Contenido (extracto del texto, aunque no está claro en este HTML, se puede dejar vacío o capturar algo más si es necesario)
            content = None

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Autor': author,
                'Fecha Publicación': date,
                'Categorías': 'Estudios',  # Campo fijo basado en el ejemplo
                'URL Imagen': image_url,
                'Contenido': content,
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

def extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los artículos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana). Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)  # Inicializar el WebDriver solo una vez
    current_page = start_page
    all_articles_df = pd.DataFrame()  # Inicializamos un DataFrame vacío

    while current_page <= max_pages:
        # Construir la URL de la página actual
        url = f"{base_url}{current_page}/"  # Aquí el formato de la URL, el número de página se añade al final
        print(f"Extrayendo artículos de: {url}")

        # Llamar a la función extract_articles_from_url2 para obtener artículos de la página
        all_articles_df = extract_articles_from_url2(driver, url, all_articles_df)

        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_articles_df is None or len(all_articles_df) == 0:
            print(f"No se encontraron artículos en la página {current_page}, o la página no existe.")
            break

        current_page += 1  # Pasar a la siguiente página

    driver.quit()  # Cerrar el WebDriver después de todas las iteraciones
    return all_articles_df

In [29]:
base_url = "https://institutolibertad.cl/publicaciones-instituto-libertad/"
ins_lib_pub = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=22, headless=False)

Extrayendo artículos de: https://institutolibertad.cl/publicaciones-instituto-libertad/1/
Extrayendo artículos de: https://institutolibertad.cl/publicaciones-instituto-libertad/2/
Extrayendo artículos de: https://institutolibertad.cl/publicaciones-instituto-libertad/3/
Extrayendo artículos de: https://institutolibertad.cl/publicaciones-instituto-libertad/4/
Extrayendo artículos de: https://institutolibertad.cl/publicaciones-instituto-libertad/5/
Extrayendo artículos de: https://institutolibertad.cl/publicaciones-instituto-libertad/6/
Extrayendo artículos de: https://institutolibertad.cl/publicaciones-instituto-libertad/7/
Extrayendo artículos de: https://institutolibertad.cl/publicaciones-instituto-libertad/8/
Extrayendo artículos de: https://institutolibertad.cl/publicaciones-instituto-libertad/9/
Extrayendo artículos de: https://institutolibertad.cl/publicaciones-instituto-libertad/10/
Extrayendo artículos de: https://institutolibertad.cl/publicaciones-instituto-libertad/11/
Extrayen

In [30]:
ins_lib_pub

,Título,Enlace,Autor,Fecha Publicación,Categorías,URL Imagen,Contenido,Fecha Consulta
0,Plan “Calles sin Violencia”: Un fracaso vatici...,https://institutolibertad.cl/publicaciones/pla...,No especificado,"julio 26, 2024",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:08:38
1,Informe de perspectiva económica Julio 2024,https://institutolibertad.cl/publicaciones/inf...,No especificado,"julio 19, 2024",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:08:38
2,Análisis Económico sobre la Tasa de Política M...,https://institutolibertad.cl/publicaciones/per...,No especificado,"julio 18, 2024",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:08:38
3,Informe sobre empleabilidad julio 2024,https://institutolibertad.cl/publicaciones/inf...,No especificado,"julio 13, 2024",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:08:38
4,Perspectiva Política Julio 2024,https://institutolibertad.cl/publicaciones/per...,No especificado,"julio 12, 2024",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:08:39
...,...,...,...,...,...,...,...,...
120,Comenzó la desaceleración económica: ¿Qué espe...,https://institutolibertad.cl/publicaciones/com...,No especificado,"abril 4, 2022",Estudios,None,None,2024-11-25 02:10:57
121,Material Seminario Proceso Constitucional IL 2023,https://institutolibertad.cl/publicaciones/sem...,No especificado,"marzo 3, 2022",Estudios,None,None,2024-11-25 02:10:57
122,Consecuencias de terminar con el bicameralismo...,https://institutolibertad.cl/publicaciones/con...,No especificado,"febrero 25, 2022",Estudios,None,None,2024-11-25 02:10:58
123,Situación fiscal más desafiante,https://institutolibertad.cl/publicaciones/sit...,No especificado,"febrero 8, 2022",Estudios,None,None,2024-11-25 02:10:58


In [31]:
base_url = "https://institutolibertad.cl/noticias/"
ins_lib_not = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=5, headless=False)
ins_lib_not

Extrayendo artículos de: https://institutolibertad.cl/noticias/1/
Extrayendo artículos de: https://institutolibertad.cl/noticias/2/
Extrayendo artículos de: https://institutolibertad.cl/noticias/3/
Extrayendo artículos de: https://institutolibertad.cl/noticias/4/
Extrayendo artículos de: https://institutolibertad.cl/noticias/5/


,Título,Enlace,Autor,Fecha Publicación,Categorías,URL Imagen,Contenido,Fecha Consulta
0,Trump: la opción de disuadir o intimidar,https://institutolibertad.cl/trump-la-opcion-d...,No especificado,"noviembre 10, 2024",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:11:21
1,ATENCIÓN: rendición de algunos gastos electora...,https://institutolibertad.cl/atencion-rendicio...,No especificado,"octubre 22, 2024",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:11:22
2,Centros de Estudio realizaron el seminario «Re...,https://institutolibertad.cl/centros-de-estudi...,No especificado,"octubre 21, 2024",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:11:22
3,Comisión de RREE del Instituto Libertad expone...,https://institutolibertad.cl/comision-de-rree-...,No especificado,"octubre 14, 2024",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:11:22
4,Instituto Libertad y Renovación Nacional lanza...,https://institutolibertad.cl/instituto-liberta...,No especificado,"octubre 8, 2024",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:11:22
...,...,...,...,...,...,...,...,...
65,Luis Castillo y Enrique Paris participaron de ...,https://institutolibertad.cl/luis-castillo-y-e...,No especificado,"mayo 9, 2022",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:11:50
66,En conversación con IL David Bravo señala la n...,https://institutolibertad.cl/en-conversacion-c...,No especificado,"mayo 3, 2022",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:11:50
67,Expresidente del Banco Central Rodrigo Vergara...,https://institutolibertad.cl/expresidente-del-...,No especificado,"abril 29, 2022",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:11:50
68,IL organizó en Coquimbo encuentro para explica...,https://institutolibertad.cl/il-organizo-encue...,No especificado,"abril 22, 2022",Estudios,https://institutolibertad.cl/wp-content/upload...,None,2024-11-25 02:11:50


## Iteración sobre enlaces

In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (se puede ajustar el tiempo de espera si es necesario)
    driver.implicitly_wait(3)  # Esperar hasta 3 segundos para que se cargue la página
    
    # Extraer la información de interés:
    
    try:
        # Título del artículo
        title = driver.find_element(By.CSS_SELECTOR, "h1.elementor-heading-title").text.strip()
    except:
        title = "No disponible"
    
    try:
        # Fecha de publicación
        publication_date = driver.find_element(By.CSS_SELECTOR, "span.elementor-post-info__item--type-date").text.strip()
    except:
        publication_date = "No disponible"
    
    try:
        # Contenido del artículo
        content = driver.find_element(By.CSS_SELECTOR, "div.elementor-widget-theme-post-content").text.strip()
    except:
        content = "No disponible"
    
    try:
        # Imagen destacada (si se desea extraer la imagen)
        image_url = driver.find_element(By.CSS_SELECTOR, "div.elementor-widget-image img").get_attribute("src")
    except:
        image_url = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Fecha de Publicación": publication_date,
        "Contenido": content,
        "Imagen": image_url,
        "Enlace": url,
    }
    
    return article_data

def process_all_links(jg_df, headless=True):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.
    
    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.
        headless (bool): Determina si el modo headless debe estar activado o no. Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        for _, row in jg_df.iterrows():
            url = row['Enlace']
            print(f"Extrayendo datos de: {url}")
            
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar

In [35]:
ins_lib_not2 = ins_lib_not[:2]
ins_lib_notdf = process_all_links(ins_lib_not, headless=False)
ins_lib_notdf

Extrayendo datos de: https://institutolibertad.cl/trump-la-opcion-de-disuadir-o-intimidar/
Extrayendo datos de: https://institutolibertad.cl/atencion-rendicion-de-algunos-gastos-electorales-solo-puede-ser-hasta-el-25-de-octubre/
Extrayendo datos de: https://institutolibertad.cl/centros-de-estudio-realizaron-el-seminario-reflexiones-a-cinco-anos-del-18-de-octubre/
Extrayendo datos de: https://institutolibertad.cl/comision-de-rree-del-instituto-libertad-expone-en-el-congreso-acerca-de-la-relacion-chile-bolivia/
Extrayendo datos de: https://institutolibertad.cl/instituto-libertad-y-renovacion-nacional-lanzan-chat-de-inteligencia-artificial-para-candidatos/
Extrayendo datos de: https://institutolibertad.cl/luis-pardo-conduce-panel-sobre-politicas-publicas-para-atraer-inversion-a-latinoamerica-en-el-encuentro-atlantico-de-think-tanks-2024/
Extrayendo datos de: https://institutolibertad.cl/graduacion-de-la-segunda-version-del-diplomado-en-normativa-y-gestion-municipal/
Extrayendo datos de: h

,Título,Fecha de Publicación,Contenido,Imagen,Enlace
0,Trump: la opción de disuadir o intimidar,"noviembre 10, 2024",(Columna de Rafael Aldunate en El Líbero)\n T...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/trump-la-opcion-d...
1,ATENCIÓN: rendición de algunos gastos electora...,"octubre 22, 2024",La ley N° 21.693 que estableció que las elecci...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/atencion-rendicio...
2,Centros de Estudio realizaron el seminario «Re...,"octubre 21, 2024",A cinco años de los acontecimientos del 18 de ...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/centros-de-estudi...
3,Comisión de RREE del Instituto Libertad expone...,"octubre 14, 2024",La Comisión de Relaciones Exteriores de la Cám...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/comision-de-rree-...
4,Instituto Libertad y Renovación Nacional lanza...,"octubre 8, 2024",En un esfuerzo por brindar más apoyo a los can...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/instituto-liberta...
...,...,...,...,...,...
65,Luis Castillo y Enrique Paris participaron de ...,"mayo 9, 2022","El ex ministro Enrique Paris, y el ex subsecre...",https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/luis-castillo-y-e...
66,En conversación con IL David Bravo señala la n...,"mayo 3, 2022",A siete años de haberle entregado a la expresi...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/en-conversacion-c...
67,Expresidente del Banco Central Rodrigo Vergara...,"abril 29, 2022","Rodrigo Vergara, expresidente del Banco Centra...",https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/expresidente-del-...
68,IL organizó en Coquimbo encuentro para explica...,"abril 22, 2022",Más de 60 dirigentes sociales y gremiales de l...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/il-organizo-encue...


In [38]:
ins_lib_notdf["Categoría"] = 'Noticias'
ins_lib_notdf["Think tank"] = 'Instituto Libertad'

In [36]:
ins_lib_pubdf = process_all_links(ins_lib_pub, headless=False)
ins_lib_pubdf

Extrayendo datos de: https://institutolibertad.cl/publicaciones/plan-calles-sin-violencia-un-fracaso-vaticinado-por-la-falta-de-metodo/
Extrayendo datos de: https://institutolibertad.cl/publicaciones/informe-de-perspectiva-economica-julio-2024/
Extrayendo datos de: https://institutolibertad.cl/publicaciones/perspectiva-municipal-regional-julio-2024/
Extrayendo datos de: https://institutolibertad.cl/publicaciones/informe-sobre-empleabilidad-julio-2024/
Extrayendo datos de: https://institutolibertad.cl/publicaciones/perspectiva-politica-julio-2024/
Extrayendo datos de: https://institutolibertad.cl/publicaciones/informe-imacec-julio-2024/
Extrayendo datos de: https://institutolibertad.cl/publicaciones/informe-imacec-abril-2024/
Extrayendo datos de: https://institutolibertad.cl/publicaciones/expectativas-economica-junio-2024/
Extrayendo datos de: https://institutolibertad.cl/publicaciones/empleo-mayo-2024/
Extrayendo datos de: https://institutolibertad.cl/publicaciones/perspectiva-municipa

,Título,Fecha de Publicación,Contenido,Imagen,Enlace
0,Plan “Calles sin Violencia”: Un fracaso vatici...,"julio 26, 2024",El informe de análisis del plan «Calles Sin Vi...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/pla...
1,Informe de perspectiva económica Julio 2024,"julio 19, 2024",El informe de Perspectiva Económica de julio 2...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/inf...
2,Análisis Económico sobre la Tasa de Política M...,"julio 18, 2024",El Informe de Análisis Económico sobre la Tasa...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/per...
3,Informe sobre empleabilidad julio 2024,"julio 13, 2024",El Informe de Opinión Económica sobre el Emple...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/inf...
4,Perspectiva Política Julio 2024,"julio 12, 2024",El informe de Perspectiva Política de julio 20...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/per...
...,...,...,...,...,...
120,Comenzó la desaceleración económica: ¿Qué espe...,"abril 4, 2022","En el primer mes de 2022, el Imacec registró u...",https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/com...
121,Material Seminario Proceso Constitucional IL 2023,"marzo 3, 2022",Junto con agradecer su asistencia a este semin...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/sem...
122,Consecuencias de terminar con el bicameralismo...,"febrero 25, 2022",La principal consecuencia de esta reforma serí...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/con...
123,Situación fiscal más desafiante,"febrero 8, 2022",De acuerdo a los datos más recientes del Banco...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/sit...


In [39]:
ins_lib_pubdf["Categoría"] = 'Publicaciones'
ins_lib_pubdf["Think tank"] = 'Instituto Libertad'

In [40]:
ins_lib = pd.concat([ins_lib_pubdf, ins_lib_notdf], ignore_index=True)
ins_lib

,Título,Fecha de Publicación,Contenido,Imagen,Enlace,Categoría,Think tank
0,Plan “Calles sin Violencia”: Un fracaso vatici...,"julio 26, 2024",El informe de análisis del plan «Calles Sin Vi...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/pla...,Publicaciones,Instituto Libertad
1,Informe de perspectiva económica Julio 2024,"julio 19, 2024",El informe de Perspectiva Económica de julio 2...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/inf...,Publicaciones,Instituto Libertad
2,Análisis Económico sobre la Tasa de Política M...,"julio 18, 2024",El Informe de Análisis Económico sobre la Tasa...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/per...,Publicaciones,Instituto Libertad
3,Informe sobre empleabilidad julio 2024,"julio 13, 2024",El Informe de Opinión Económica sobre el Emple...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/inf...,Publicaciones,Instituto Libertad
4,Perspectiva Política Julio 2024,"julio 12, 2024",El informe de Perspectiva Política de julio 20...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/publicaciones/per...,Publicaciones,Instituto Libertad
...,...,...,...,...,...,...,...
190,Luis Castillo y Enrique Paris participaron de ...,"mayo 9, 2022","El ex ministro Enrique Paris, y el ex subsecre...",https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/luis-castillo-y-e...,Noticias,Instituto Libertad
191,En conversación con IL David Bravo señala la n...,"mayo 3, 2022",A siete años de haberle entregado a la expresi...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/en-conversacion-c...,Noticias,Instituto Libertad
192,Expresidente del Banco Central Rodrigo Vergara...,"abril 29, 2022","Rodrigo Vergara, expresidente del Banco Centra...",https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/expresidente-del-...,Noticias,Instituto Libertad
193,IL organizó en Coquimbo encuentro para explica...,"abril 22, 2022",Más de 60 dirigentes sociales y gremiales de l...,https://institutolibertad.cl/wp-content/upload...,https://institutolibertad.cl/il-organizo-encue...,Noticias,Instituto Libertad


In [41]:
ins_lib.to_excel("Instituto Libertad.xlsx", index = False)

# Anterior

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

def extract_articles_from_url(url, existing_data=None):
    """
    Extrae información de artículos desde una URL y concatena con un DataFrame existente si se proporciona.

    Args:
        url (str): URL de la página para extraer artículos.
        existing_data (pd.DataFrame, opcional): DataFrame existente con datos previos. Default: None.

    Returns:
        pd.DataFrame: DataFrame con la información de los artículos extraídos y los datos previos si se proporciona.
    """
    def extract_articles_info(driver):
        wait = WebDriverWait(driver, 10)
        articles = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "article.col-md-4.mb-5")))

        data = []
        for article in articles:
            try:
                if article.is_displayed():
                    # Título
                    title_element = article.find_element(By.CSS_SELECTOR, ".tarjeta__titulo a")
                    title = title_element.text
                    link = title_element.get_attribute("href")

                    # Autor
                    author_element = article.find_element(By.CSS_SELECTOR, ".tarjeta__autor a")
                    author = author_element.text

                    # Fecha
                    date_element = article.find_element(By.CSS_SELECTOR, ".tarjeta-fecha__texto")
                    date = date_element.text

                    # Categorías (no está directamente disponible en el HTML proporcionado, si es necesario se puede modificar)
                    categories = "Estudios"  # Si las categorías son siempre "Estudios", lo dejamos fijo

                    # Imagen
                    image_element = article.find_element(By.CSS_SELECTOR, "img.img-fluid")
                    image_url = image_element.get_attribute("src")

                    # Resumen o contenido inicial
                    content_element = article.find_element(By.CSS_SELECTOR, ".tarjeta__texto")
                    content = content_element.text

                    # Fecha de consulta
                    current_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                    data.append({
                        'Título': title,
                        'Enlace': link,
                        'Autor': author,
                        'Fecha Publicación': date,
                        'Categorías': categories,
                        'URL Imagen': image_url,
                        'Contenido': content,
                        'Fecha Consulta': current_date
                    })

            except Exception as e:
                print(f"Error al extraer datos de un artículo: {e}")

        return pd.DataFrame(data)

    # Inicialización del WebDriver
    wd = webdriver.Chrome()
    try:
        wd.get(url)
        WebDriverWait(wd, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "article.col-md-4.mb-5")))
        new_data = extract_articles_info(wd)
        
        # Concatenar con datos existentes si se proporciona
        if existing_data is not None:
            result_data = pd.concat([existing_data, new_data], ignore_index=True)
        else:
            result_data = new_data

        return result_data

    finally:
        wd.quit()

Apuntes constitucionales

In [2]:
url = "https://horizontalchile.cl/publicaciones/"
df = extract_articles_from_url(url)
df

Error al extraer datos de un artículo: Message: no such element: Unable to locate element: {"method":"css selector","selector":".tarjeta__autor a"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7314C6CB5+28821]
	(No symbol) [0x00007FF731433840]
	(No symbol) [0x00007FF7312D578A]
	(No symbol) [0x00007FF7313291BE]
	(No symbol) [0x00007FF7313294AC]
	(No symbol) [0x00007FF73131C52C]
	(No symbol) [0x00007FF73134F33F]
	(No symbol) [0x00007FF73131C3F6]
	(No symbol) [0x00007FF73134F510]
	(No symbol) [0x00007FF73136F412]
	(No symbol) [0x00007FF73134F0A3]
	(No symbol) [0x00007FF73131A778]
	(No symbol) [0x00007FF73131B8E1]
	GetHandleVerifier [0x00007FF7317FFCAD+3408013]
	GetHandleVerifier [0x00007FF73181741F+3504127]
	GetHandleVerifier [0x00007FF73180B5FD+3455453]
	GetHandleVerifier [0x00007FF73158BDBB+835995]
	(No symbo

,Título,Enlace,Autor,Fecha Publicación,Categorías,URL Imagen,Contenido,Fecha Consulta
0,El estallido social chileno visto como un «eve...,https://horizontalchile.cl/publicacion/el-esta...,Juan Luis Ossa,18 Oct 2024,Estudios,https://horizontalchile.cl/assets/uploads/2024...,"A cinco años del estallido social, el document...",2024-11-25 00:42:14
1,Análisis del presupuesto en seguridad 2024-2025,https://horizontalchile.cl/publicacion/analisi...,Alfonso España,17 Oct 2024,Estudios,https://horizontalchile.cl/assets/uploads/2024...,El siguiente documento hace una revisión del n...,2024-11-25 00:42:14
2,Financiamiento Público de la Educación Superio...,https://horizontalchile.cl/publicacion/financi...,Santiago Montiel,15 Oct 2024,Estudios,https://horizontalchile.cl/assets/uploads/2024...,Según estimaciones del centro de estudios Hori...,2024-11-25 00:42:14
3,Análisis crítico de la ley de estabilización d...,https://horizontalchile.cl/publicacion/analisi...,Soledad Hormazábal,23 Sep 2024,Estudios,https://horizontalchile.cl/assets/uploads/2024...,Resulta muy preocupante que se realice polític...,2024-11-25 00:42:15
4,"Two Journeys, Twice the Learning: 10 takeaways...",https://horizontalchile.cl/publicacion/two-jou...,"Hernán Larraín M.,",11 Sep 2024,Estudios,https://horizontalchile.cl/assets/uploads/2024...,The authors explain that the guidelines and ac...,2024-11-25 00:42:15


In [ ]:
url = "https://www.fjguzman.cl/apuntes-constitucionales/page/2/"
df = extract_articles_from_url(url, existing_data=df)
df

Revista realidad

In [ ]:
url = "https://www.fjguzman.cl/revista-realidad/"
df = extract_articles_from_url(url, existing_data=df)
df

Taller de coyuntura

In [ ]:
url = "https://www.fjguzman.cl/taller-de-coyuntura/"
df = extract_articles_from_url(url, existing_data=df)
df

In [ ]:
url = "https://www.fjguzman.cl/taller-de-coyuntura/page/2/"
df = extract_articles_from_url(url, existing_data=df)
df.head()

In [ ]:
url = "https://www.fjguzman.cl/taller-de-coyuntura/page/3/"
df = extract_articles_from_url(url, existing_data=df)
df

## Resto

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

def extract_articles_from_url2(url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    driver = webdriver.Chrome()
    
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)
        
        # Obtener todos los artículos
        articles = driver.find_elements(By.CLASS_NAME, "post-entry")
        
        data = []
        for article in articles:
            # Imagen
            try:
                img = article.find_element(By.CSS_SELECTOR, ".post-featured-image img")
                image_url = img.get_attribute("src")
            except:
                image_url = None
                
            # Título y enlace
            try:
                title = article.find_element(By.CLASS_NAME, "post-entry__title").text.strip()
                link = article.find_element(By.CSS_SELECTOR, ".post-entry__more a").get_attribute("href")
            except:
                continue  # Si no hay título o enlace, saltamos este artículo
                
            # Contenido
            try:
                content = article.find_element(By.CSS_SELECTOR, ".post-entry__content p").text.strip()
            except:
                content = None
            
            # Fecha actual de consulta
            current_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Autor': 'No especificado',  # Campo fijo
                'Fecha Publicación': 'No especificada',  # Campo fijo
                'Categorías': 'Ideas & Propuestas',  # Campo fijo
                'URL Imagen': image_url,
                'Contenido': content,
                'Fecha Consulta': current_date
            }
            
            data.append(article_data)
        
        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)
        
        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df
        
        return existing_df
        
    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()
        
    finally:
        driver.quit()

In [ ]:
url = "https://www.fjguzman.cl/ideas-propuestas/page/2/"
aver = extract_articles_from_url2(url, aver)
aver

In [ ]:
aver

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime

def extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=100):
    """
    Itera sobre varias páginas y extrae los artículos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas procesadas.
    """
    current_page = start_page
    all_articles_df = pd.DataFrame()  # Inicializamos un DataFrame vacío
    
    while current_page <= max_pages:
        # Construir la URL de la página actual
        url = base_url.format(page_number=current_page)
        print(f"Extrayendo artículos de: {url}")
        
        # Llamar a la función extract_articles_from_url2 para obtener artículos de la página
        all_articles_df = extract_articles_from_url2(url, all_articles_df)
        
        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_articles_df is None or len(all_articles_df) == 0:
            print(f"No se encontraron artículos en la página {current_page}, o la página no existe.")
            break
        
        current_page += 1  # Pasar a la siguiente página
    
    return all_articles_df

In [ ]:
# URL base de la página, con el marcador {page_number} para indicar el número de página
base_url = "https://www.fjguzman.cl/ideas-propuestas/page/{page_number}/"

# Llamamos a la función para extraer artículos de todas las páginas
aver = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=50)

# Ver el DataFrame con los artículos extraídos
aver

In [ ]:
jg = pd.concat([df, aver], ignore_index=True)
jg

Mirada política

In [ ]:
# URL base de la página, con el marcador {page_number} para indicar el número de página
base_url = "https://www.fjguzman.cl/mirada-politica/page/{page_number}/"
mp = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=15)
mp

In [ ]:
jg = pd.concat([jg, mp], ignore_index=True)
jg

Libros

In [ ]:
# URL base de la página, con el marcador {page_number} para indicar el número de página
base_url = "https://www.fjguzman.cl/libros/page/{page_number}/"
libros = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=15)
libros

In [ ]:
jg = pd.concat([jg, libros], ignore_index=True)
jg

Infografías

In [ ]:
# URL base de la página, con el marcador {page_number} para indicar el número de página
base_url = "https://www.fjguzman.cl/infografias/page/{page_number}/"
infog = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=3)
infog

In [ ]:
jg = pd.concat([jg, infog], ignore_index=True)
jg

Videos

In [ ]:
# URL base de la página, con el marcador {page_number} para indicar el número de página
base_url = "https://www.fjguzman.cl/videos/page/{page_number}/"
videos = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=7)
videos

In [ ]:
jg = pd.concat([jg, videos], ignore_index=True)
jg

Columnas

In [ ]:
# URL base de la página, con el marcador {page_number} para indicar el número de página
base_url = "https://www.fjguzman.cl/columnas/page/{page_number}/"
cols = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=36)
cols

In [ ]:
jg = pd.concat([jg, cols], ignore_index=True)
jg

In [ ]:
jg.to_excel("FundacionJaimeGuzman.xlsx", index=False)

# Iteración sobre links

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo desde una URL usando un webdriver existente.

    Args:
        driver (webdriver): El webdriver ya abierto.
        url (str): URL de la página para extraer información.

    Returns:
        dict: Diccionario con la información extraída del artículo.
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)
        
        # Obtener el artículo
        article = driver.find_element(By.CSS_SELECTOR, "article.post")
        
        # Extraer los datos del artículo
        article_data = {}
        
        # Título
        try:
            article_data['Título'] = article.find_element(By.CLASS_NAME, "entry-title").text.strip()
        except:
            article_data['Título'] = None
        
        # Autor
        try:
            article_data['Autor'] = article.find_element(By.CSS_SELECTOR, ".byline .author .fn.n").text.strip()
        except:
            article_data['Autor'] = None
        
        # Fecha de publicación
        try:
            date_str = article.find_element(By.CSS_SELECTOR, ".posted-on time").get_attribute("datetime")
            article_data['Fecha Publicación'] = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S+00:00").strftime('%Y-%m-%d %H:%M:%S')
        except:
            article_data['Fecha Publicación'] = None
        
        # Categorías
        try:
            categories = [cat.text.strip() for cat in article.find_elements(By.CSS_SELECTOR, ".cat-links a")]
            article_data['Categorías'] = ", ".join(categories) if categories else None
        except:
            article_data['Categorías'] = None
        
        # Contenido
        try:
            article_data['Contenido'] = article.find_element(By.CLASS_NAME, "entry-content").text.strip()
        except:
            article_data['Contenido'] = None
        
        # Enlace del artículo
        article_data['Enlace'] = url
        
        # Fecha de consulta (cuando se extrajo la información)
        article_data['Fecha Consulta'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        return article_data
    
    except Exception as e:
        print(f"Error en la extracción de {url}: {str(e)}")
        return None

In [ ]:
jg.Categorías.unique()

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

def process_all_links(jg_df):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.

    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.

    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        for _, row in jg_df.iterrows():
            url = row['Enlace']
            print(f"Extrayendo datos de: {url}")
            
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar

In [ ]:
jg_info = process_all_links(jg)
jg_info

In [ ]:
jg_info2 = jg_info[:3]
jg_info2

In [ ]:
# Guardar el DataFrame como JSON en un archivo de texto
jg_info.to_json("FundacionJaimeGuzman_COMPLETO.json", orient='records', lines=True)

print("Archivo guardado como .json")

In [ ]:
import pandas as pd

# Leer el archivo JSON por línea (JSON de registros)
uu = pd.read_json("FundacionJaimeGuzman_COMPLETO.json", lines=True)

# Ver los primeros registros
uu

In [ ]:
jg_info.columns

In [ ]:
jg_info["Medio de comunicación"] = "Fundación Jaime Guzmán"

jg_info  = jg_info[['Fecha Publicación', 'Autor','Categorías', 'Título', 'Medio de comunicación', 'Enlace', 'Contenido']]

In [ ]:
# Cambiar nombres de las columnas
jg_info = jg_info.rename(columns={
    'Fecha Publicación': 'Fecha',
    'Autor': 'Área',
    'Categorías': 'Categoria',
    'Título': 'Nombre de la noticia',
    'Medio de comunicación': 'Think tank',
    'Contenido': 'Corpus'
})

In [ ]:
jg_info

In [ ]:
import pandas as pd
import re

# Reemplazar caracteres ilegales en el DataFrame
def clean_illegal_characters(df):
    # Usar una expresión regular para eliminar caracteres de control
    return df.applymap(lambda x: re.sub(r'[\x00-\x1F\x7F]', '', str(x)) if isinstance(x, str) else x)

# Limpiar el DataFrame de caracteres ilegales
jg_info_clean = clean_illegal_characters(jg_info)

# Guardar el DataFrame limpio como archivo .xlsx
jg_info_clean.to_excel("FundacionJaimeGuzman_COMPLETO.xlsx", index=False)

print("Archivo guardado como .xlsx")

In [ ]:
jg_info_clean

In [ ]:
jg_info_clean

In [ ]:
import pandas as pd
import re

# Función para extraer autor, medio y fecha
def extract_metadata(row):
    # Intentamos la primera estructura: "Por Autor\nMedio, Fecha"
    match1 = re.match(r"^(Por\s)(.*?)\n(.*?), (\d{1,2} de \w+ \d{4})", row['Corpus'])
    if match1:
        autor = match1.group(2)  # Autor está después de "Por "
        medio = match1.group(3)  # Medio está antes de la coma
        fecha = match1.group(4)  # Fecha está después de la coma
        # Crear el corpus2 eliminando autor, medio y fecha del principio
        corpus2 = re.sub(r"^(Por\s" + re.escape(autor) + r"\n" + re.escape(medio) + r", " + re.escape(fecha) + r"\n)", "", row['Corpus'])
    else:
        # Intentamos la segunda estructura: "Texto largo... Medio, Fecha al final"
        match2 = re.search(r"(.*?), (.*?), (\d{1,2} de \w+ \d{4})$", row['Corpus'])
        if match2:
            autor = match2.group(1)  # Autor está antes de la primera coma
            medio = match2.group(2)  # Medio está entre las comas
            fecha = match2.group(3)  # Fecha está después de la última coma
            # Crear el corpus2 eliminando autor, medio y fecha del final
            corpus2 = re.sub(r"\s?" + re.escape(autor) + r",\s?" + re.escape(medio) + r",\s?" + re.escape(fecha) + r"$", "", row['Corpus'])
        else:
            autor = None
            medio = None
            fecha = None
            corpus2 = row['Corpus']

    return pd.Series([autor, medio, fecha, corpus2])

# Crear una copia del DataFrame original
jg_info2 = jg_info.copy()

# Aplicar la función a las filas del DataFrame
jg_info2[['Autor', 'Medio de comunicación', 'Fecha de publicación', 'Corpus2']] = jg_info2.apply(extract_metadata, axis=1)

# Paso 1: Eliminar "Por " del autor si existe
jg_info2['Autor'] = jg_info2['Autor'].str.replace(r'^Por ', '', regex=True)

# Paso 2: Convertir la fecha a formato datetime (sin hora)
jg_info2['Fecha de publicación'] = pd.to_datetime(jg_info2['Fecha de publicación'], format='%d de %B %Y').dt.date

jg_info2

In [ ]:
# Limpiar el DataFrame de caracteres ilegales
jg_info_clean2 = clean_illegal_characters(jg_info2)

# Guardar el DataFrame limpio como archivo .xlsx
jg_info_clean2.to_excel("FundacionJaimeGuzman_COMPLETOv2.xlsx", index=False)

print("Archivo guardado como .xlsx")

# Ideas Republicanas

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import json

def extract_links_from_gallery(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Buscar todos los elementos de la galería
    gallery_items = soup.find_all('div', class_='eael-filterable-gallery-item-wrap')
    if not gallery_items:
        return pd.DataFrame()
    
    # Extraer información de cada item
    data = []
    for item in gallery_items:
        # Buscar el enlace
        link_tag = item.find('a', href=True)
        if link_tag:
            link = link_tag['href']
            
            # Buscar el título
            title_tag = item.find('h5', class_='fg-item-title')
            title = title_tag.text.strip() if title_tag else ''
            
            # Buscar la fecha
            date_tag = item.find('p')
            date = date_tag.text.strip() if date_tag else ''
            
            # Guardar los datos
            data.append({
                'titulo': title,
                'enlace': link,
                'fecha': date
            })
    
    # Crear DataFrame
    df = pd.DataFrame(data)
    
    # Limpiar los datos
    df['titulo'] = df['titulo'].str.strip()
    df['fecha'] = df['fecha'].str.strip()
    
    return df

# Leer el contenido del archivo HTML
with open('ideas_rep1.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Crear el DataFrame
df = extract_links_from_gallery(html_content)

# Mostrar el resultado
print(df.to_string(index=False))

In [ ]:
df

In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup

def extract_links_from_gallery_from_folder(folder_path):
    # Crear un DataFrame vacío para concatenar los datos
    all_data = []
    
    # Obtener todos los archivos HTML en la carpeta
    html_files = [f for f in os.listdir(folder_path) if f.endswith('.html')]
    
    if not html_files:
        print("No se encontraron archivos HTML en la carpeta.")
        return pd.DataFrame()
    
    # Iterar sobre cada archivo HTML
    for file_name in html_files:
        file_path = os.path.join(folder_path, file_name)
        
        # Leer el contenido del archivo HTML
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        # Mostrar el progreso en consola
        print(f"Procesando el archivo: {file_name}")
        
        # Extraer los datos del archivo
        df = extract_links_from_gallery(html_content)
        
        if not df.empty:
            # Agregar una columna con el nombre del archivo para distinguir el origen
            df['archivo_origen'] = file_name
            all_data.append(df)

        print(len(df))
    
    # Concatenar todos los DataFrames de los archivos
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        print("No se extrajeron datos de ningún archivo.")
        return pd.DataFrame()

def extract_links_from_gallery(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Buscar los elementos de la galería o columnas
    gallery_items = soup.find_all('div', class_=['eael-filterable-gallery-item-wrap', 'elementor-column'])
    
    # También buscar los encabezados que contienen enlaces
    heading_items = soup.find_all('div', class_='elementor-widget-heading')
    
    if not gallery_items and not heading_items:
        return pd.DataFrame()
    
    data = []

    # Extraer los enlaces y demás datos de los elementos de galería
    for item in gallery_items:
        link_tag = item.find('a', href=True)
        if link_tag:
            link = link_tag['href']
            
            title_tag = item.find(['h5', 'h3', 'h4'], class_='fg-item-title')
            title = title_tag.text.strip() if title_tag else ''
            
            date_tag = item.find(['h4', 'p'])
            date = date_tag.text.strip() if date_tag else ''
            
            data.append({
                'titulo': title,
                'enlace': link,
                'fecha': date
            })
    
    # Extraer los enlaces de los encabezados que contienen un enlace
    for item in heading_items:
        link_tag = item.find('a', href=True)
        if link_tag:
            link = link_tag['href']
            title = link_tag.text.strip()
            data.append({
                'titulo': title,
                'enlace': link,
                'fecha': ''  # No hay fecha asociada a estos elementos
            })
    
    # Crear DataFrame
    df = pd.DataFrame(data)
    
    # Limpiar los datos
    df['titulo'] = df['titulo'].str.strip()
    df['fecha'] = df['fecha'].str.strip()
    
    return df

    
    # Crear DataFrame
    df = pd.DataFrame(data)
    
    # Limpiar los datos
    df['titulo'] = df['titulo'].str.strip()
    df['fecha'] = df['fecha'].str.strip()
    
    return df

# Llamada a la función para procesar los archivos en una carpeta
folder_path = r"D:\Escritorio D\Scrap Rodrigo\Ideas Republicanas"  # Sustituye por la ruta de tu carpeta
df_final = extract_links_from_gallery_from_folder(folder_path)

df_final

## Otra prueba

In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup

def extract_links_from_gallery_from_folder(folder_path):
    # Crear un DataFrame vacío para concatenar los datos
    all_data = []
    
    # Obtener todos los archivos HTML en la carpeta
    html_files = [f for f in os.listdir(folder_path) if f.endswith('.html')]
    
    if not html_files:
        print("No se encontraron archivos HTML en la carpeta.")
        return pd.DataFrame()
    
    # Iterar sobre cada archivo HTML
    for file_name in html_files:
        file_path = os.path.join(folder_path, file_name)
        
        # Leer el contenido del archivo HTML
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        # Mostrar el progreso en consola
        print(f"Procesando el archivo: {file_name}")
        
        # Extraer los datos del archivo
        df = extract_links_from_gallery(html_content)
        
        if not df.empty:
            # Agregar una columna con el nombre del archivo para distinguir el origen
            df['archivo_origen'] = file_name
            all_data.append(df)

        print(len(df))
    
    # Concatenar todos los DataFrames de los archivos
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        print("No se extrajeron datos de ningún archivo.")
        return pd.DataFrame()

def extract_links_from_gallery(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Buscar los elementos de galería o columnas
    gallery_items = soup.find_all('div', class_=['eael-filterable-gallery-item-wrap', 'elementor-column'])
    
    # Buscar los elementos con encabezados que contienen enlaces (en widgets o botones)
    heading_items = soup.find_all('div', class_='elementor-widget-heading')
    image_items = soup.find_all('div', class_='elementor-widget-image')
    button_items = soup.find_all('div', class_='elementor-widget-button')
    
    # Buscar específicamente los títulos <h3> con clase 'elementor-heading-title'
    h3_titles = soup.find_all('h3', class_='elementor-heading-title')

    if not gallery_items and not heading_items and not image_items and not button_items and not h3_titles:
        return pd.DataFrame()
    
    data = []

    # Extraer los enlaces de los elementos de galería
    for item in gallery_items:
        link_tag = item.find('a', href=True)
        if link_tag:
            link = link_tag['href']
            
            title_tag = item.find(['h5', 'h3', 'h4'], class_='fg-item-title')
            title = title_tag.text.strip() if title_tag else ''
            
            date_tag = item.find(['h4', 'p'])
            date = date_tag.text.strip() if date_tag else ''
            
            data.append({
                'titulo': title,
                'enlace': link,
                'fecha': date
            })
    
    # Extraer los enlaces de los encabezados (títulos) y sus posibles enlaces
    for item in heading_items:
        link_tag = item.find('a', href=True)
        if link_tag:
            link = link_tag['href']
            title = link_tag.text.strip()
            data.append({
                'titulo': title,
                'enlace': link,
                'fecha': ''  # No hay fecha asociada a estos elementos
            })

    # Extraer enlaces de las imágenes con su enlace asociado
    for item in image_items:
        link_tag = item.find('a', href=True)
        if link_tag:
            link = link_tag['href']
            # Puedes optar por usar el texto alternativo de la imagen o su URL
            title = item.find('img')['alt'] if item.find('img') else ''
            data.append({
                'titulo': title,
                'enlace': link,
                'fecha': ''  # Sin fecha asociada
            })

    # Extraer enlaces de los botones
    for item in button_items:
        link_tag = item.find('a', href=True)
        if link_tag:
            link = link_tag['href']
            title = item.find('span', class_='elementor-button-text').text.strip() if item.find('span', class_='elementor-button-text') else ''
            data.append({
                'titulo': title,
                'enlace': link,
                'fecha': ''  # No hay fecha asociada
            })
    
    # Extraer títulos directamente desde los <h3 class="elementor-heading-title">
    for item in h3_titles:
        title = item.text.strip()
        link_tag = item.find('a', href=True)
        if link_tag:
            link = link_tag['href']
            data.append({
                'titulo': title,
                'enlace': link,
                'fecha': ''  # No hay fecha asociada a estos elementos
            })
    
    # Crear DataFrame
    df = pd.DataFrame(data)
    
    # Limpiar los datos
    df['titulo'] = df['titulo'].str.strip()
    df['fecha'] = df['fecha'].str.strip()
    
    return df

# Llamada a la función para procesar los archivos en una carpeta
folder_path = r"D:\Escritorio D\Scrap Rodrigo\Ideas Republicanas"  # Sustituye por la ruta de tu carpeta
df_final = extract_links_from_gallery_from_folder(folder_path)

df_final

In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup

def extract_links_from_gallery_from_folder(folder_path):
    # Crear un DataFrame vacío para concatenar los datos
    all_data = []
    
    # Obtener todos los archivos HTML en la carpeta
    html_files = [f for f in os.listdir(folder_path) if f.endswith('.html')]
    
    if not html_files:
        print("No se encontraron archivos HTML en la carpeta.")
        return pd.DataFrame()
    
    # Iterar sobre cada archivo HTML
    for file_name in html_files:
        file_path = os.path.join(folder_path, file_name)
        
        # Leer el contenido del archivo HTML
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        # Mostrar el progreso en consola
        print(f"Procesando el archivo: {file_name}")
        
        # Extraer los datos del archivo
        df = extract_links_from_gallery(html_content)
        
        if not df.empty:
            # Agregar una columna con el nombre del archivo para distinguir el origen
            df['archivo_origen'] = file_name
            all_data.append(df)

        print(f"Cantidad de elementos extraídos: {len(df)}")
    
    # Concatenar todos los DataFrames de los archivos
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        print("No se extrajeron datos de ningún archivo.")
        return pd.DataFrame()

def extract_links_from_gallery(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Buscar todos los elementos de columna
    column_items = soup.find_all('div', class_='elementor-column')
    
    if not column_items:
        return pd.DataFrame()
    
    data = []

    # Iterar sobre cada columna
    for item in column_items:
        # Extraer el enlace de la imagen
        image_link = ''
        image_title = ''
        image_tag = item.find('div', class_='elementor-widget-image')
        if image_tag:
            link_tag = image_tag.find('a', href=True)
            if link_tag:
                image_link = link_tag['href']
            img_tag = image_tag.find('img', alt=True)
            if img_tag:
                image_title = img_tag['alt']
        
        # Extraer la fecha (si está presente)
        date = ''
        date_tag = item.find('h4', class_='elementor-heading-title')
        if date_tag:
            date_link_tag = date_tag.find('a', href=True)
            if date_link_tag:
                date = date_link_tag.text.strip()
        
        # Extraer el título (si está presente)
        title = ''
        title_tag = item.find('h3', class_='elementor-heading-title')
        if title_tag:
            title = title_tag.text.strip()

        # Extraer el enlace del botón (si está presente)
        button_link = ''
        button_text = ''
        button_tag = item.find('div', class_='elementor-widget-button')
        if button_tag:
            button_a_tag = button_tag.find('a', href=True)
            if button_a_tag:
                button_link = button_a_tag['href']
            button_span_tag = button_tag.find('span', class_='elementor-button-text')
            if button_span_tag:
                button_text = button_span_tag.text.strip()

        # Agregar los datos extraídos a la lista
        data.append({
            'titulo': title,
            'enlace_imagen': image_link,
            'titulo_imagen': image_title,
            'fecha': date,
            'titulo_boton': button_text,
            'enlace_boton': button_link
        })
    
    # Crear DataFrame con los datos extraídos
    df = pd.DataFrame(data)
    
    # Limpiar los datos
    df['titulo'] = df['titulo'].str.strip()
    df['fecha'] = df['fecha'].str.strip()
    df['titulo_imagen'] = df['titulo_imagen'].str.strip()
    df['titulo_boton'] = df['titulo_boton'].str.strip()

    return df


# Llamada a la función para procesar los archivos en una carpeta
folder_path = r"D:\Escritorio D\Scrap Rodrigo\Ideas Republicanas"  # Sustituye por la ruta de tu carpeta
df_final1 = extract_links_from_gallery_from_folder(folder_path)

df_final1

In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup

def extract_links_from_gallery_from_folder(folder_path):
    # Crear un DataFrame vacío para concatenar los datos
    all_data = []
    
    # Obtener todos los archivos HTML en la carpeta
    html_files = [f for f in os.listdir(folder_path) if f.endswith('.html')]
    
    if not html_files:
        print("No se encontraron archivos HTML en la carpeta.")
        return pd.DataFrame()
    
    # Iterar sobre cada archivo HTML
    for file_name in html_files:
        file_path = os.path.join(folder_path, file_name)
        
        # Leer el contenido del archivo HTML
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        # Mostrar el progreso en consola
        print(f"Procesando el archivo: {file_name}")
        
        # Extraer los datos del archivo
        df = extract_links_from_gallery(html_content)
        
        if not df.empty:
            # Agregar una columna con el nombre del archivo para distinguir el origen
            df['archivo_origen'] = file_name
            all_data.append(df)

        print(len(df))
    
    # Concatenar todos los DataFrames de los archivos
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        print("No se extrajeron datos de ningún archivo.")
        return pd.DataFrame()

def extract_links_from_gallery(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Buscar los elementos de galería de filtro con la clase 'eael-filterable-gallery-item-wrap'
    gallery_items = soup.find_all('div', class_='eael-filterable-gallery-item-wrap')
    
    if not gallery_items:
        return pd.DataFrame()
    
    data = []

    # Extraer los enlaces y títulos de los elementos de la galería
    for item in gallery_items:
        # Buscar el enlace
        link_tag = item.find('a', href=True)
        if link_tag:
            link = link_tag['href']
            
            # Buscar el título dentro de <h5 class="fg-item-title">
            title_tag = item.find('h5', class_='fg-item-title')
            title = title_tag.text.strip() if title_tag else ''
            
            # Buscar la fecha dentro de <p class="fg-item-content">
            date_tag = item.find('p', class_='fg-item-content')
            date = date_tag.text.strip() if date_tag else ''
            
            # Buscar la imagen
            image_tag = item.find('img', class_='gallery-item-thumbnail')
            image_url = image_tag['src'] if image_tag else ''
            
            data.append({
                'titulo': title,
                'enlace': link,
                'fecha': date,
                'enlace_imagen': image_url
            })
    
    # Crear DataFrame con los datos extraídos
    df = pd.DataFrame(data)
    
    # Limpiar los datos
    df['titulo'] = df['titulo'].str.strip()
    df['fecha'] = df['fecha'].str.strip()
    df['enlace_imagen'] = df['enlace_imagen'].str.strip()

    return df


# Llamada a la función para procesar los archivos en una carpeta
folder_path = r"D:\Escritorio D\Scrap Rodrigo\Ideas Republicanas"  # Sustituye por la ruta de tu carpeta
df_final = extract_links_from_gallery_from_folder(folder_path)

df_final

In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup

def extract_links_from_gallery_from_folder(folder_path):
    # Crear un DataFrame vacío para concatenar los datos
    all_data = []
    
    # Obtener todos los archivos HTML en la carpeta
    html_files = [f for f in os.listdir(folder_path) if f.endswith('.html')]
    
    if not html_files:
        print("No se encontraron archivos HTML en la carpeta.")
        return pd.DataFrame()
    
    # Iterar sobre cada archivo HTML
    for file_name in html_files:
        file_path = os.path.join(folder_path, file_name)
        
        # Leer el contenido del archivo HTML
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        # Mostrar el progreso en consola
        print(f"Procesando el archivo: {file_name}")
        
        # Extraer los datos del archivo
        df = extract_links_from_gallery(html_content)
        
        if not df.empty:
            # Agregar una columna con el nombre del archivo para distinguir el origen
            df['archivo_origen'] = file_name
            all_data.append(df)

        print(len(df))
    
    # Concatenar todos los DataFrames de los archivos
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        print("No se extrajeron datos de ningún archivo.")
        return pd.DataFrame()

def extract_links_from_gallery(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Buscar todos los elementos de columna
    column_items = soup.find_all('div', class_='elementor-column')
    
    if not column_items:
        return pd.DataFrame()
    
    data = []

    # Iterar sobre cada columna
    for item in column_items:
        # Extraer los enlaces de imagen
        image_link = ''
        image_title = ''
        image_tag = item.find('div', class_='elementor-widget-image')
        if image_tag:
            link_tag = image_tag.find('a', href=True)
            if link_tag:
                image_link = link_tag['href']
            img_tag = image_tag.find('img', alt=True)
            if img_tag:
                image_title = img_tag['alt']
        
        # Extraer la fecha (si está presente)
        date = ''
        date_tag = item.find('h4', class_='elementor-heading-title')
        if date_tag:
            date_link_tag = date_tag.find('a', href=True)
            if date_link_tag:
                date = date_link_tag.text.strip()
        
        # Extraer el título (si está presente)
        title = ''
        title_tag = item.find('h3', class_='elementor-heading-title')
        if title_tag:
            title = title_tag.text.strip()

        # Extraer el enlace del botón (si está presente)
        button_link = ''
        button_text = ''
        button_tag = item.find('div', class_='elementor-widget-button')
        if button_tag:
            button_a_tag = button_tag.find('a', href=True)
            if button_a_tag:
                button_link = button_a_tag['href']
            button_span_tag = button_tag.find('span', class_='elementor-button-text')
            if button_span_tag:
                button_text = button_span_tag.text.strip()

        # Agregar los datos extraídos a la lista
        data.append({
            'titulo': title,
            'enlace_imagen': image_link,
            'titulo_imagen': image_title,
            'fecha': date,
            'titulo_boton': button_text,
            'enlace_boton': button_link
        })
    
    # Crear DataFrame con los datos extraídos
    df = pd.DataFrame(data)
    
    # Limpiar los datos
    df['titulo'] = df['titulo'].str.strip()
    df['fecha'] = df['fecha'].str.strip()
    df['titulo_imagen'] = df['titulo_imagen'].str.strip()
    df['titulo_boton'] = df['titulo_boton'].str.strip()

    return df


# Llamada a la función para procesar los archivos en una carpeta
folder_path = r"D:\Escritorio D\Scrap Rodrigo\Ideas Republicanas"  # Sustituye por la ruta de tu carpeta
df_final = extract_links_from_gallery_from_folder(folder_path)

df_final

## Iterar sobre enlaces

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from datetime import datetime

def extract_info_from_url(driver, url):
    """
    Extrae el contenido y el autor de un artículo desde una URL usando un webdriver existente.

    Args:
        driver (webdriver): El webdriver ya abierto.
        url (str): URL de la página para extraer información.

    Returns:
        dict: Diccionario con la información extraída del artículo.
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)
        
        # Obtener el artículo
        article = driver.find_element(By.CSS_SELECTOR, "div.entry-content.clear")
        
        # Extraer los datos del artículo
        article_data = {}
        
        # Contenido del artículo (Corpus)
        try:
            corpus = ' '.join([p.text.strip() for p in article.find_elements(By.TAG_NAME, 'p')])
            article_data['Corpus'] = corpus
        except:
            article_data['Corpus'] = None
        
        # Autor (buscar los <strong> y la estructura asociada)
        try:
            # Buscar todos los párrafos en el artículo
            paragraphs = article.find_elements(By.TAG_NAME, "p")
            author_name = None

            # Buscar autor dentro de <strong> en los párrafos
            for para in paragraphs:
                # Ver si hay un <strong> dentro de este párrafo
                strong_tags = para.find_elements(By.TAG_NAME, "strong")
                if strong_tags:
                    # Si encontramos un <strong> que parece el autor, tomamos el primer <strong>
                    author_name = strong_tags[0].text.strip()
                    break
            
            # Si no se encuentra en el formato esperado, intentar de otras formas
            if not author_name:
                # Intentar buscar el nombre del autor en cualquier otro lugar específico
                author_paragraphs = article.find_elements(By.XPATH, "//p[strong]")
                if author_paragraphs:
                    # Si hay un párrafo con <strong>, obtener el primer <strong> como autor
                    author_name = author_paragraphs[0].find_element(By.TAG_NAME, "strong").text.strip()
            
            # Guardar el autor si se encontró
            article_data['Autor'] = author_name if author_name else None

        except Exception as e:
            print(f"Error al extraer el autor: {str(e)}")
            article_data['Autor'] = None
        
        return article_data
    
    except Exception as e:
        print(f"Error en la extracción de {url}: {str(e)}")
        return None

In [ ]:

def process_all_links(df):
    """
    Procesa todos los enlaces de un DataFrame 'df', extrayendo el contenido y el autor de cada artículo
    y almacenándolos en las columnas 'Corpus' y 'Autor' del DataFrame.

    Args:
        df (pd.DataFrame): DataFrame que contiene una columna 'enlace' con los enlaces de los artículos.

    Returns:
        pd.DataFrame: DataFrame con las columnas 'Corpus' y 'Autor' actualizadas.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)
    #driver = webdriver.Chrome()

    # Crear listas para almacenar los datos de los artículos
    corpus_list = []
    autor_list = []

    try:
        # Recorrer los enlaces en el DataFrame
        for _, row in df.iterrows():
            url = row['enlace']
            print(f"Extrayendo datos de: {url}")
            
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                corpus_list.append(article_data.get('Corpus', ''))
                autor_list.append(article_data.get('Autor', ''))
            else:
                corpus_list.append('')
                autor_list.append('')
        
        # Actualizar el DataFrame con los datos extraídos
        df['Corpus'] = corpus_list
        df['Autor'] = autor_list
        
        return df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return df
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar


# Ejemplo de uso
# Suponiendo que df_final es el DataFrame con los enlaces a procesar
#df_final2 = df_final[:3]
df_final22 = process_all_links(df_final)

# Mostrar el DataFrame actualizado
df_final22

In [ ]:
def process_all_links(df):
    """
    Procesa todos los enlaces de un DataFrame 'df', extrayendo el contenido y el autor de cada artículo
    y almacenándolos en las columnas 'Corpus' y 'Autor' del DataFrame.

    Args:
        df (pd.DataFrame): DataFrame que contiene una columna 'enlace' con los enlaces de los artículos.

    Returns:
        pd.DataFrame: DataFrame con las columnas 'Corpus' y 'Autor' actualizadas.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear listas para almacenar los datos de los artículos
    corpus_list = []
    autor_list = []

    try:
        # Total de filas a procesar
        total_rows = len(df)

        # Recorrer los enlaces en el DataFrame
        for index, row in df.iterrows():
            url = row['enlace']
            print(f"Extrayendo datos de: {url}")

            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)

            # Almacenar los datos extraídos
            if article_data:
                corpus_list.append(article_data.get('Corpus', ''))
                autor_list.append(article_data.get('Autor', ''))
            else:
                corpus_list.append('')
                autor_list.append('')

            # Mostrar el progreso en cada iteración
            print(f"Progreso: {index + 1}/{total_rows} ({(index + 1) / total_rows * 100:.2f}%)")
            
            # Mostrar los primeros 20 caracteres del corpus y el autor
            print(f"Fila procesada - Autor: {article_data.get('Autor', 'N/A')}, Corpus: {article_data.get('Corpus', '')[:20]}")
        
        # Actualizar el DataFrame con los datos extraídos
        df['Corpus'] = corpus_list
        df['Autor'] = autor_list
        
        return df

    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return df

    finally:
        driver.quit()  # Cerrar el webdriver al finalizar


# Ejemplo de uso
# Suponiendo que df_final es el DataFrame con los enlaces a procesar
#df_final2 = df_final[:3]  # Puedes probar con un subconjunto si lo deseas
df_final22 = process_all_links(df_final)

# Mostrar el DataFrame actualizado
df_final22

## Unión

In [ ]:
df_final.columns

In [ ]:
df_final['Autor'] = ""
df_final["Medio de comunicación"] = ""
df_final["Think tank"] = "Ideas Republicanas"

df_final  = df_final[['fecha', 'archivo_origen', 'titulo', 'Think tank', 'enlace_imagen', 'Autor', 'titulo_imagen', 'titulo_boton',
       'enlace_boton']]

# Cambiar nombres de las columnas
df_final = df_final.rename(columns={
    'fecha': 'Fecha',
    'archivo_origen': 'Categoria',
    'titulo': 'Nombre de la noticia',
    'enlace_imagen': 'Enlace'
})

In [ ]:
df_final22.columns

In [ ]:
df_final22["Think tank"] = "Ideas Republicanas"

df_final22  = df_final22[['fecha', 'archivo_origen', 'titulo', 'Think tank', 'enlace', 'enlace_imagen', 'Autor', 'Corpus']]

# Cambiar nombres de las columnas
df_final22 = df_final22.rename(columns={
    'fecha': 'Fecha',
    'archivo_origen': 'Categoria',
    'titulo': 'Nombre de la noticia',
    'enlace_imagen': 'Enlace'
})

In [ ]:
df_final.columns

In [ ]:
df_final22.columns

In [ ]:
df_combinado = pd.concat([df_final, df_final22], ignore_index=True, sort=False)
df_combinado

In [ ]:
df_combinado.to_excel("Ideas Republicanas.xlsx", index = False)

# aparte